In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.utils.data as Data
import torchvision
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import numpy as np
%matplotlib inline
import torch.optim as optim

In [2]:
torch.manual_seed(1)

In [3]:
EPOCH = 10
BATCH_SIZE = 32
LR = 0.005
path = '/Users/ruchira/Documents/lvl_4_project/lvl4-hons-project/notebooks/train_test_imbal/'

In [4]:
X_train = np.loadtxt(path + 'X_train.csv', delimiter=',')
y_train = np.loadtxt(path + 'y_train.csv', delimiter=',')
X_test = np.loadtxt(path + 'X_test.csv', delimiter=',')
y_test = np.loadtxt(path + 'y_test.csv', delimiter=',')

In [5]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(109381, 430)
(109381,)
(36461, 430)
(36461,)


In [6]:
train_arr = np.hstack((X_train, y_train.reshape(-1,1)))
print(train_arr.shape)

(109381, 431)


In [7]:
tensor_Xtrain = torch.from_numpy(X_train)
tensor_ytrain = torch.from_numpy(y_train)

train_data = Data.TensorDataset(tensor_Xtrain, tensor_ytrain)
train_loader = Data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)

In [8]:
class Encoder(nn.Module):
    def __init__(self, D_in, H, D_out): #why is H=1??? not set anywhere why is it needed, check original impl
        super(Encoder, self).__init__()
        modules = []
        hidden_dims = [32,64,128,256,512] # second 32 coming from this, should we change this? try with only one hidden layer
        # no for loop just give it one value and see
        print("D in", D_in)
        print("H", H)
        print("D out", D_out)
        for h_dim in hidden_dims:
            modules.append(
                nn.Sequential(
                    nn.Conv1d(D_in, 1, kernel_size=2, stride=1, padding=1),
                    nn.BatchNorm1d(h_dim),
                    nn.LeakyReLU()
                    ))
            in_channels = h_dim
        self.encoder = nn.Sequential(*modules)
#         print(h_dim)
#         self.linear1 = nn.Linear(D_in, H)
#         self.linear2 = nn.Linear(H, D_out)
        
    def forward(self,x):
#         x = F.relu(self.linear1(x))
#         return F.relu(self.linear2(x))
        return self.encoder(x)

class Decoder(nn.Module):
    def __init__(self, D_in, H, D_out):
        super(Decoder, self).__init__()
#         self.linear1 = torch.nn.Linear(D_in, H)
#         self.linear2 = torch.nn.Linear(H, D_out)
        hidden_dims = [32,64,128,256,512]
        modules = []
        self.decoder_input = nn.Linear(8, hidden_dims[-1]*4)
        hidden_dims.reverse()
        
        for i in range(len(hidden_dims)-1):
            modules.append(
                nn.Sequential(
                    nn.ConvTranspose1d(hidden_dims[i],
                                hidden_dims[i+1],
                                kernel_size=3,
                                stride = 2,
                                padding=1,
                                output_padding=1),
                    nn.BatchNorm1d(hidden_dims[i+1]), 
                    nn.LeakyReLU()
                ))
        
        self.decoder = nn.Sequential(*modules)
        self.final_layer = nn.Sequential(
                            nn.ConvTranspose1d(hidden_dims[-1],
                                              hidden_dims[-1],
                                              kernel_size=3,
                                              stride=2,
                                              padding=1,
                                              output_padding=1),
                            nn.BatchNorm1d(hidden_dims[-1]),
                            nn.LeakyReLU(),
                            nn.Conv1d(hidden_dims[-1], out_channels=3,
                                     kernel_size=3, padding=1),
                            nn.Tanh())
    def forward(self, x):
        x = self.decoder_input(x)
        x = x.view(-1,430)
        x = self.decoder(x)
        return self.final_layer(x)
    
class VAE(nn.Module):
    latent_dim = 8

    def __init__(self, encoder, decoder):
        super(VAE, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
#         self.latent_dim = 8
        self._enc_mu = torch.nn.Linear(1024, 8)
        self._enc_log_sigma = torch.nn.Linear(1024, 8)
    def _sample_latent(self, h_enc):
        """
        Return the latent normal sample z ~ N(mu, sigma^2)
        """
        mu = self._enc_mu(h_enc)
        log_sigma = self._enc_log_sigma(h_enc)
        sigma = torch.exp(log_sigma)
        std_z = torch.from_numpy(np.random.normal(0, 1, size=sigma.size())).float()

        self.z_mean = mu
        self.z_sigma = sigma

        return mu + sigma * Variable(std_z, requires_grad=False)  # Reparameterization trick

    def forward(self, state):
        h_enc = self.encoder(state)
        z = self._sample_latent(h_enc)
        return self.decoder(z)


def latent_loss(z_mean, z_stddev):
    mean_sq = z_mean * z_mean
    stddev_sq = z_stddev * z_stddev
    return 0.5 * torch.mean(mean_sq + stddev_sq - torch.log(stddev_sq) - 1)


In [9]:
view_data = Variable(tensor_Xtrain[:5].view(-1, 430).type(torch.FloatTensor)/255.)

In [10]:
input_dim = 1 * 430
batch_size = 32

# encoder = Encoder(input_dim, 100, 100)
# decoder = Decoder(8, 100, input_dim)
# vae = VAE(encoder, decoder)


print('Number of samples: ', len(train_data))
print(input_dim)
encoder = Encoder(batch_size,1,input_dim)
decoder = Decoder(2, 1024, input_dim)
vae = VAE(encoder, decoder)

criterion = nn.MSELoss()

optimizer = optim.Adam(vae.parameters(), lr=0.0001)
l = None
for epoch in range(10):
    for i, data in enumerate(train_loader, 0):
        inputs, classes = data
        inputs, classes = Variable(inputs.resize_(batch_size, 1, input_dim)).type(torch.FloatTensor), Variable(classes).type(torch.FloatTensor)
        print(inputs.shape)
        optimizer.zero_grad()
        dec = vae(inputs)
        ll = latent_loss(vae.z_mean, vae.z_sigma)
        loss = criterion(dec, inputs) + ll
        loss.backward()
        optimizer.step()
        l = loss.item()
    print(epoch, l)
    vae_data = vae(view_data)
    f, a = plt.subplots(2, 5, figsize=(20,5))

    for i in range(5):
        a[0][i].plot(view_data.data.numpy()[i])

    for i in range(5):
        a[1][i].clear()
        a[1][i].plot(vae_data.data.numpy()[i])
    plt.show();


Number of samples:  109381
430
D in 32
H 1
D out 430
torch.Size([32, 1, 430])


RuntimeError: Given groups=1, weight of size [1, 32, 2], expected input[32, 1, 430] to have 32 channels, but got 1 channels instead